### Math 5364 - Data mining I - Tarleton State Univ - Dr. Scott Cook - Fall 2019
Assignment 9 - due 2019/10/31

Run Naive Bayes Classifier on the HouseVotes84 data set found here:

https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/




Get the data

In [0]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

--2019-10-23 11:32:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-10-23 11:32:23 (277 KB/s) - ‘house-votes-84.data’ saved [18171/18171]

--2019-10-23 11:32:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6868 (6.7K) [application/x-httpd-php]
Saving to: ‘house-votes-84.names’

house-votes-84.name 100%[======

Import into pandas

In [0]:
import numpy as np
import pandas as pd

df = pd.read_csv(f"/content/house-votes-84.data", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


From .names file, here are the columns

7. Attribute Information:
   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)

  Let's put these onto the columns

In [0]:
bills = ['party',
         'infants',
         'water',
         'budget',
         'physician',
         'elsalvador',
         'religious',
         'satellite',
         'contras',
         'missile',
         'immigration',
         'synfuels',
         'education',
         'superfund',
         'crime',
         'duty-free',
         'south-africa',
         ]
df.columns = bills
df.head()

,party,infants,water,budget,physician,elsalvador,religious,satellite,contras,missile,immigration,synfuels,education,superfund,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df.replace({'y':True, 'n':False, '?':np.nan}, inplace=True)
df.head()

,party,infants,water,budget,physician,elsalvador,religious,satellite,contras,missile,immigration,synfuels,education,superfund,crime,duty-free,south-africa
0,republican,False,True,False,True,True,True,False,False,False,True,NaN,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,NaN
2,democrat,NaN,True,True,NaN,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,NaN,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,NaN,True,True,True,True


In [0]:
X = df.iloc[:,1:]
X.head()

,infants,water,budget,physician,elsalvador,religious,satellite,contras,missile,immigration,synfuels,education,superfund,crime,duty-free,south-africa
0,False,True,False,True,True,True,False,False,False,True,NaN,True,True,True,False,True
1,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,NaN
2,NaN,True,True,NaN,True,True,False,False,False,False,True,False,True,True,False,False
3,False,True,True,False,NaN,True,False,False,False,False,True,False,True,False,False,True
4,True,True,True,False,True,True,False,False,False,False,True,NaN,True,True,True,True


In [0]:
Y = df['party']
Y.head()
Y.value_counts()

democrat      267
republican    168
Name: party, dtype: int64

We'd like to apply sci-kit learn's Naive Bayes (the Bernoulli one since the features and targets are bool).  But we get errors due to the "?" entries.

In [0]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
model.fit(X,Y)

ValueError: ignored

So, we need to decide how to impute these missing values.  This is always delicate.  We need to think carefully about what missing data means *before* we adopt a scheme to impute.

As stated in the .names file, each ? could mean
- voted present
- voted present to avoid conflict of interest
- did not vote or otherwise make a position known

I don't fully understand all the nuances here, but it seems clear that these mean different things.  My own interpretations are
- voted present - "I don't have an opinion"
- voted present to avoid conflict of interest - "I have a personal stake in the outcome, so I should not cast a vote"
- did not vote or otherwise make a position known - absent, sick, etc

I may be wrong here, but I'm pretty confident that ? is fairly broad here.

Assignment: Build a Naive Bayes Classifier using Sci-Kit Learn's BernoulliNB.  Use the following:

- k=10 fold cross-validation
- Evaluation criteria = classification accuracy on validation set (proportion of model party predictions on the validation set that are correct)
- Tune the following TWO hyperparameters:
  - Imputation method (FOUR options)
    - row mean: Fill with most frequent vote cast by the congressperson
    - column mean: Fill with most frequent vote cast on that bill
    - with party (2 variants).  Let p = proportion of member's party that voted "Y" for that bill.
      - with party deterministic:  member votes "Y" iff p >= 0.5
      - with party probabilistic:  member votes "Y" with probability p
  - BernoulliNB's alpha parameter
    - I'm not sure how important this will be, nor what a reasonable range of options is.  Do some reading & playing.
    - Focus on imputation method.  I think that one will be more critical.


Remember to cross-validate within EACH loop of hyperparameter tuning.  So, for each choice of (imputation method, alpha), you get k=10 classification accuracy scores.  Take the average.  This is the "model performance" for that (imputation method, alpha).


Report the optimal choice for (imputation method, alpha) and its model performance.

Here's a little code to get you started on "with party" imputation.

In [0]:
# groupby party and compute yes vote proportion
yes_prop = X.groupby(Y).agg(lambda x : x.mean())

In [0]:
# This gives rows for dem and rep, columns for each BILL.
# For curiosity, I'd also like the overall yes vote proportion.
# In pandas, it's easier to create new columns than new rows.
# So, I'll transpose so I can add an "all" columns (rather than "all" row")
yes_prop = yes_prop.T
yes_prop['all'] = X.mean(axis=0)
yes_prop

party,democrat,republican,all
infants,0.604651,0.187879,0.442080
water,0.502092,0.506757,0.503876
budget,0.888462,0.134146,0.596698
physician,0.054054,0.987879,0.417453
elsalvador,0.215686,0.951515,0.504762
religious,0.476744,0.897590,0.641509
satellite,0.772201,0.240741,0.567696
contras,0.828897,0.152866,0.576190
missile,0.758065,0.115152,0.501211
immigration,0.471483,0.557576,0.504673


Suggestions for with party imputation: produce "imputation votes" for EVERY entry of X, but only actually use them where X has nan

- use the first line of code above, but replace agg -> transform so that yes_prop.shape = X.shape
- use that to get "imputation votes" for ALL entries of X
- copy "imputation votes" into X ONLY for entries with nan.

Yes, this is wasteful because it throws away most of the imputation votes.  But I doubt you will notice any effect on performance.